In [1]:
from seirsplus.models import *
import networkx
from datetime import datetime
import numpy as np
from hyperopt import hp
from functools import partial

In [2]:
model = SEIRSModel(beta=0.155, sigma=1/5.2, gamma=1/12.39, initN=100000, initI=100)

In [3]:
numNodes = 10000
baseGraph    = networkx.barabasi_albert_graph(n=numNodes, m=9)
G_normal     = custom_exponential_graph(baseGraph, scale=100)
# Social distancing interactions:
G_distancing = custom_exponential_graph(baseGraph, scale=10)
# Quarantine interactions:
G_quarantine = custom_exponential_graph(baseGraph, scale=5)

model = SEIRSNetworkModel(G=G_normal, beta=0.155, sigma=1/5.2, gamma=1/12.39, mu_I=0.0004, p=0.5,
                          Q=G_quarantine, beta_D=0.155, sigma_D=1/5.2, gamma_D=1/12.39, mu_D=0.0004,
                          theta_E=0.02, theta_I=0.02, phi_E=0.2, phi_I=0.2, psi_E=1.0, psi_I=1.0, q=0.5,
                          initI=10)

checkpoints = {'t': [20, 100], 'G': [G_distancing, G_normal], 'p': [0.1, 0.5], 'theta_E': [0.02, 0.02], 'theta_I': [0.02, 0.02], 'phi_E':   [0.2, 0.2], 'phi_I':   [0.2, 0.2]}

model.run(T=300, checkpoints=checkpoints)

model.figure_infections()

t = 0.06
t = 10.01
[Checkpoint: Updating parameters]
t = 20.55
t = 30.15
t = 40.00
t = 50.09
t = 60.06
t = 70.01
t = 80.00
t = 90.00
[Checkpoint: Updating parameters]
t = 100.00
t = 110.00
t = 120.00
t = 130.00
t = 140.00
t = 150.01
t = 160.00
t = 170.04
t = 180.00
t = 190.04
t = 200.04
t = 210.05
t = 220.09
t = 230.31
t = 240.12
t = 250.10
t = 260.06
t = 270.22
t = 280.72
t = 290.22


<Figure size 1200x800 with 1 Axes>

In [4]:
S = model.numS
t = model.tseries 
I = model.numI

In [5]:
len(S), len(t)

(23617, 23617)

## Tune Hyperparameters

In [113]:
#### Since the model outputs a different format than the CSSEGISandData, we need to align them
#### which means - finding the appropriate prediction for the model for a particular date

### modelI - Time Series Prediction output by model (model.numI)
### modelT - Time Series of the model (model.tseries)
### dates  - dates in the true file that we have from CSSEGISandData - datetime object


def alignTimeSeries(modelI, modelT, dates):
#     datesS = ['3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20']
#     dates = []
#     for date in datesS:
#         dates.append(datetime.strptime(date, "%m/%d/%y"))
    finalModelPredictions = []
    count = 0
    day0 = dates[0]
    for date in dates:
        t = (date-day0).days
        print(t)
        while(modelT[count]<=t):
            count+=1
            if(count == len(modelT)):
                print("Last prediction reached - Number of predictions less than required")
                finalModelPredictions.append(modelI[count-1])
                return finalModelPredictions
            
        diff1 = t - modelT[count-1]
        diff2 = modelT[count] - t
        if(diff1<=diff2):
            finalModelPredictions.append(modelI[count-1])
            count -=1
        else:
            finalModelPredictions.append(modelI[count])
            
    return finalModelPredictions

In [114]:
Itemp = I[:18]
Ttemp = t[:18]
for i in range(18):
    print(Itemp[i], Ttemp[i], i)

(10.0, 0.0, 0)
(10.0, 0.686695632541538, 1)
(10.0, 0.7608766577585226, 2)
(9.0, 1.1376487929365617, 3)
(8.0, 1.2552836905769018, 4)
(8.0, 1.6429438860912144, 5)
(8.0, 1.6761708760031906, 6)
(8.0, 2.4372343433385133, 7)
(9.0, 2.7426277698730845, 8)
(8.0, 3.0899739340926975, 9)
(7.0, 3.2277154167139064, 10)
(6.0, 3.3526440771815773, 11)
(6.0, 3.405332060716196, 12)
(6.0, 4.155704358851042, 13)
(7.0, 4.16572756439204, 14)
(6.0, 4.184063949048825, 15)
(6.0, 4.254265622475148, 16)
(5.0, 6.209814830593387, 17)


In [115]:
predtemp = alignTimeSeries(Itemp, Ttemp, None)

0
1
2
3
4


In [116]:
print(predtemp)

[10.0, 9.0, 8.0, 8.0, 6.0]


In [107]:
#Trying out hyperOpt to check how it works - ignore this

def objective(params):
    a = params['a'] 
    b = params['b']
    print(a, b, a-b)
    return a - b

# define a search space
from hyperopt import hp
space = {'a': hp.uniform('a', -10, 10), 
         'b': hp.uniform('b', 0, 10)}

# minimize the objective over the space
from hyperopt import fmin, tpe, space_eval
best = fmin(objective, space, algo=tpe.suggest, max_evals=5)
print(space_eval(space, best))

(-5.817031359452336, 6.161028104798607, -11.978059464250943)
(-2.5276109169958456, 6.146726281191109, -8.674337198186954)
(-3.167512330472791, 4.882532677798338, -8.050045008271129)                
(3.7638167536884275, 1.6441156654226408, 2.1197010882657867)               
(-2.893467285655169, 6.1149546087162685, -9.008421894371438)                
100%|██████████| 5/5 [00:00<00:00, 69.08trial/s, best loss: -11.9780594643] 
{'a': -5.817031359452336, 'b': 6.161028104798607}


In [ ]:
def tuneParameters(params, actualFile):
#     numNodes = 10000
#     baseGraph    = networkx.barabasi_albert_graph(n=numNodes, m=9)
#     G_normal     = custom_exponential_graph(baseGraph, scale=100)
#     # Social distancing interactions:
#     G_distancing = custom_exponential_graph(baseGraph, scale=10)
#     # Quarantine interactions:
#     G_quarantine = custom_exponential_graph(baseGraph, scale=5)

#     model = SEIRSNetworkModel(G=G_normal, beta=0.155, sigma=1/5.2, gamma=1/12.39, mu_I=0.0004, p=0.5,
#                               Q=G_quarantine, beta_D=0.155, sigma_D=1/5.2, gamma_D=1/12.39, mu_D=0.0004,
#                               theta_E=0.02, theta_I=0.02, phi_E=0.2, phi_I=0.2, psi_E=1.0, psi_I=1.0, q=0.5,
#                               initI=10)

#     checkpoints = {'t': [20, 100], 'G': [G_distancing, G_normal], 'p': [0.1, 0.5], 'theta_E': [0.02, 0.02], 'theta_I': [0.02, 0.02], 'phi_E':   [0.2, 0.2], 'phi_I':   [0.2, 0.2]}

#     model.run(T=300, checkpoints=checkpoints)

#TODO - get forecast from models after reading parameters from 'params'
    
    modelPredictions = model.numI
    modelTseries = model.tseries
    
    modelPredictionsAlign = alignTimeSeries(modelPredictions, modelTseries, dates)
    
    return loss(modelPredictions, trueValues, 'rmse')

In [ ]:
search_space = {'a': hp.uniform('a', -10, 10), 
         'b': hp.uniform('b', 0, 10)}

partialTuneParameters = partial(tuneParameters, actualFile = './path-to-actual-file')
best = fmin(tuneParameters, search_space, algo=tpe.suggest, max_evals=5)
print(space_eval(space, best))